In [27]:
import os
import requests
import json
import openai
import pandas as pd
import time
openai.api_key = "a077dc5501894130a1e69037b1e905c9"
openai.api_base = "https://ya2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-08-01-preview'

In [23]:
data = pd.read_csv('data/slot_question.csv',index_col=0)
data.head()

,ID,Question Stem,Choices,Complete Question,Answer Key,slot_question
0,8-343,A person wants to start saving money so that t...,(A) make more phone calls (B) quit eating lunc...,A person wants to start saving money so that t...,B,A person wants to start saving money so that t...
1,1129,There is most likely going to be fog around:,(A) a marsh (B) a tundra (C) the plains (D) a ...,There is most likely going to be fog around: (...,A,There is most likely going to be fog around: (...
2,880,Predators eat,(A) lions (B) humans (C) bunnies (D) grass,Predators eat (A) lions (B) humans (C) bunnies...,C,Predators eat (A) lions (B) humans (C) bunnies...
3,7-999,Oak tree seeds are planted and a sidewalk is p...,(A) roots may be split (B) roots may begin to ...,Oak tree seeds are planted and a sidewalk is p...,C,Oak tree seeds are planted and a sidewalk is p...
4,8-464,An electric car runs on electricity via,(A) gasoline (B) a power station (C) electrica...,An electric car runs on electricity via (A) ga...,C,An electric car runs on electricity via (A) ga...


In [24]:
ans_list = {}

In [28]:
for index, row in data[18:].iterrows():
    init_prompt = "Help me complete the missing option based on OpenbookQA benchmark of A,B,C,D in the sentence.\nHere are some examples:\nSentence: Frilled sharks and angler fish live far beneath the surface of the ocean, which is why they are known as (A) Deep sea animals (B) fish (C) (D) Far Sea Animals\nAnswer:(C) Long Sea Fish\n"
    prompt = init_prompt + f"Sentece: {row['slot_question']}\nAnswer:"
    try:
        response = openai.ChatCompletion.create(
            engine="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ]
        )
    except openai.error.RateLimitError:
            time.sleep(10)
            response = openai.ChatCompletion.create(
            engine="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ]
        )
            

    print(index,response['choices'][0]['message']['content'])
    ans_list[row['slot_question']] = response['choices'][0]['message']['content']

18 (D) Human
19 (C) Herbivore
20 (D) Miami
21 (B) Plastic
22 (B) Flying
23 (D) Environmental degradation
24 (C) create dazzling reflections
25 (D) its mouth
26 (B) a wheel with new brake pads
27 (D) the respiratory system
28 (C) changing shape
29 (C) Air
30 (B) Nutrient-rich soil
31 (B) The eye color of their parents.
32 (A) Deer
33 (D) Plants will grow faster if they are watered more frequently
34 (C) Volcanic activity
35 (B) Frog larvae
36 (B) the apple in the sunny yard
37 (A) Eyes
38 (B) Soil samples
39 (D) Freezer
40 (A) Reused
41 (C) Gestation period
42 (B) Industrial waste dumped in a river
43 (C) fertilization
44 (B) Measure body temperature
45 (C) Someone would light a fire
46 (D) they pop
47 (D) Explode
48 (D) Tropical
49 (B) Alive
50 (D) a bear or bird
51 (A) Quarterly
52 (C) lead to environmental degradation
53 (B) Regular exercise and balanced diet
54 (D) eye color
55 (D) Cargo Ship
56 (A) Carrots
57 (B) the nature park will be replaced by a shopping center
58 (A) Night
59

In [29]:
data['slot_answer'] = ans_list.values()

In [30]:
data.to_csv("result/mc_gpt_4.csv")